# Gradio OpenAI Chatbot

## Install the required packages

In [ ]:
%pip install -q python-dotenv==1.0.1 openai==1.35.9 gradio==4.39.0 pyautogen==0.2.32

## Import the required packages and create the Azure OpenAI client

In [ ]:
import os
from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv("../../.env")

model = os.getenv("GPT_MODEL")
endpoint=os.getenv("ENDPOINT")
api_key=os.getenv("API_KEY")
api_version=os.getenv("API_VERSION")

## Config an Autogen conversable agent

In [ ]:
config_list = [
    {
        "base_url": endpoint,
        "api_key": api_key,
        "model": model,
        "api_type": "azure",
        "api_version": api_version
    }
]

llm_config = {
    "model": model,
    "temperature": 0,
    "config_list": config_list,    
    "cache_seed": None,
}

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

## Test the agent with history

In [ ]:
messages=[
    {"role": "user","content": "List one restaurant in Miami."},
    {"role": "assistant","content": "One popular restaurant in Miami is Joe's Stone Crab."},
    {"role": "user","content": "Line one more."},
    
]
reply = agent.generate_reply(messages)
reply

## Create the gradio chatbot and have a chat using Autogen agent

In [ ]:
import gradio as gr

# This could be turned into an api: @app.post("/api/chat")
def predict(message, history):
   history_openai_format = []
   for human, assistant in history:
      history_openai_format.append({"role": "user", "content": human })
      history_openai_format.append({"role": "assistant", "content":assistant})
   history_openai_format.append({"role": "user", "content": message})
   reply = agent.generate_reply(history_openai_format)
   return reply

gr.ChatInterface(predict).launch()